In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
instructions = """
Crie uma configuração de cores que seja minimalista e divertida para que um dia bem frio seja alegre e descontraído.
"""

In [ ]:
prompt = """
Tenho a seguinte lista de lâmpadas e quero que você crie uma configuração de cores para elas que componham um tema de acordo com as seguintes instruções:

<<INSTRUÇÕES>>
{text}


Aqui está a lista de minhas lâmpadas com seus respectivos posicionamentos:

<<LISTA DE LÂMPADAS E INFORMAÇÕES RELACIONADAS>>

O teto tem o pé direito com aproximadamente 2,3 metros de altura.

Lâmpada 1: Teto, luminária esquerda, posição direita.
Lâmpada 2: Teto, luminária esquerda, posição esquerda.
Lâmpada 3: Teto, luminária direita, posição esquerda.
Lâmpada 4: Teto, luminária direita, posição direita.

Hue Iris: Mesa do meu lado direito, altura de 65cm.

Hue Play 1: Atrás do meu monitor principal do lado esquerdo.
Hue Play 2: Atrás do meu monitor principal do lado direito.

Fita Led: Na minha frente e por trás de minha mesa e 20cm, atrás do meu monitor principal, a 75cm de altura do chão.

Led cima: Na minha frente, 20cm trás do meu monitor principal, a 1,50cm de altura do chão.

<<INFORMAÇÕES DO ARQUIVO DE SAÍDA>>

Você deverá criar uma estrutura de arquivo JSON com as seguintes informações:

name: O nome da configuração. Este é um identificador único para a configuração. Crie um nome que faça sentido com as cores que compõem a configuração. O nome deve ser em inglês e caso tenha espaços em branco então troque-os para underline.

description: Uma breve descrição da configuração. Isto pode incluir detalhes sobre o ambiente em que a configuração é usada.

settings: Uma lista de configurações de luz. Cada configuração de luz deve incluir:

light_name: O nome da luz ou luminária.

color: A cor da luz. Isto deve ser um dicionário com os seguintes campos:
  red: A intensidade do vermelho (entre 0 e 255).
  green: A intensidade do verde (entre 0 e 255).
  blue: A intensidade do azul (entre 0 e 255).
  brightness: O brilho da luz (entre 0 e 255).

<<EXEMPLO DE ARQUIVO JSON>>

Aqui está um exemplo de como você pode criar uma nova configuração de luz:

Seguem os detalhes extraídos:

name: A chave 'name' representa o nome dado para a configuração atual do sistema de iluminação, que neste caso é 'sua_nova_configuracao'.

description: A chave 'description' fornece uma descrição detalhada do ambiente e das luzes, que ajuda a entender o propósito e o contexto da configuração.

settings: A chave 'settings' é uma lista que contém as configurações de cada lâmpada ou luminária que faz parte do sistema de iluminação. Cada item nesta lista é um objeto que contém duas chaves: 'light_name' e 'color'.

light_name: A chave 'light_name' é o nome dado à lâmpada ou luminária. Você deverá utilizar os nomes exatos fornecidos nestas instruções, na lista de lâmpadas.

color: A chave 'color' é um objeto que contém as informações de cor e brilho para a luz. As cores são representadas por três chaves: 'red', 'green' e 'blue', que seguem o modelo de cor RGB. A chave 'brightness' determina o nível de brilho da luz. No exemplo fornecido, ambas as luzes têm a cor definida como branco (255,255,255 no modelo RGB) e o brilho definido como 100.

Para adicionar mais configurações de lâmpadas, você pode simplesmente adicionar mais objetos à lista de 'settings'. Cada objeto deve conter as chaves 'light_name' e 'color' com as respectivas informações.

Substitua "sua_nova_configuracao", "Uma descrição detalhada do ambiente e das luzes", "Nome da sua lâmpada ou luminária 1" e "Nome da sua lâmpada ou luminária 2" pelos valores desejados. Ajuste os valores de red, green, blue e brightness para atingir a cor e o brilho desejados para cada luz.

<<SAÍDA DA RESPOSTA>>
Formate a saída como JSON e suas respectivas chaves e valores. Apenas isto.
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(prompt)
print(prompt_template)

In [ ]:
from langchain.chat_models import ChatOpenAI

messages = prompt_template.format_messages(text=instructions)
chat = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")
response = chat(messages)
print(response.content)

In [ ]:
import json

new_data = json.loads(response.content)

In [ ]:
new_data

In [ ]:
type(new_data)

In [ ]:
from marvin_hue.basics import LightConfig

lc = LightConfig().from_dict(new_data)

In [ ]:
import os
from marvin_hue.controllers import HueController

hue = HueController(ip_address=os.environ["bridge_ip"])

In [ ]:
hue.apply_light_config(light_config=lc)

In [ ]:
result = response.content

In [ ]:
from services.prompts import PromptResponseHue

prompt_result = ChatPromptTemplate.from_template(PromptResponseHue.get())
msgs = prompt_result.format_messages(instructions=instructions, text=result)
chat = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")
resp = chat(msgs)
print(resp.content)
